In [4]:
import re
dt{#}::LaTeXForm("\partial_{t}").

I::ImaginaryI.

\partial{#}::PartialDerivative;
D{#}::Derivative;


{i,j,k,l,m,n,p,q,r,s,t}::Indices(name="internal",position=independent,values={1,2,3}).

{a,b,c,d,e,f,g,h}::Indices(name="position",position=independent,values={1,2,3}).

\delta{#}::KroneckerDelta.
\epsilon{#}::EpsilonTensor(delta=\delta).


{E^{i a},A^{i}_{a},L,\beta^{a},\theta^{i},\psi^{i j}, S^{i a},\phi^{i}}::Depends(D{#},\partial{#});

${}\text{Property PartialDerivative attached to}\partial{\#}.$

${}\text{Property Derivative attached to}D{\#}.$

${}\text{Property Depends attached to}\left[E^{i a}, A^{i}\,_{a}, L, \beta^{a}, \theta^{i}, \psi^{i j}, S^{i a}, \phi^{i}\right].$

In [14]:
def trim_whitespace(string):
    ret_string = ""
    for c in string:
        if c != " ": ret_string += c
        if len(ret_string) > 0:
            if c == " " and ret_string[-1] != " ": ret_string += c
    return ret_string

def add_mult_symbols(line):
    new_line = ""
    
    in_bracket = 0
    for j in range(len(line)):
        if line[j] in "+-": new_line += " "+line[j] + " "
        else: new_line += line[j]
            
        if line[j] in "{[(": in_bracket = 1
        if line[j] in "}])": in_bracket = 0
            
        if j+1 < len(line) and line[j] not in "+-/= " and in_bracket == 0:
            if line[j+1] == " ":
                k = 1
                while k < len(line):
                    if line[j+k] not in "+/-*= ":
                        new_line += " * "
                        break
                    elif line[j+k] != " ":
                        break
                    k+=1
                j = j+k-1
    return trim_whitespace(new_line)
    
def apply_regexs_2(line,sols_dict):
    regexs = []
    subs   = []

    regexs.append(r"F\^\{(\d) (\d)\}")
    subs.append("F[(\g<2> - 1) + 3*(\g<1> - 1)]")

    regexs.append(r"(\d)(\w)")
    subs.append("\g<1> * \g<2>")
    
    regexs.append(r"S\^\{(\d) (\d)\}")
    subs.append("S[(\g<2> - 1) + 3*(\g<1> - 1)]")
    
    # Two Component Tensors
    
    regexs.append(r"\\partial_\{1\}\(([^\W_]+)[\^\{\_]+(\d)[\^\{\}\_\s]+(\d)\}\)")
    subs.append("DX(u,\g<1> + 3*(\g<2>-1) + \g<3>-1+I,dx)")
    
    regexs.append(r"\\partial_\{2\}\(([^\W_]+)[\^\{\_]+(\d)[\^\{\}\_\s]+(\d)\}\)")
    subs.append("DY(u,\g<1> + 3*(\g<2>-1) + \g<3>-1+I,dy)")
    
    regexs.append(r"\\partial_\{3\}\(([^\W_]+)[\^\{\_]+(\d)[\^\{\}\_\s]+(\d)\}\)")
    subs.append("DZ(u,\g<1> + 3*(\g<2>-1) + \g<3>-1+I,dz)")
    
    # One Component Tensors
    
    regexs.append(r"\\partial_\{1\}\(([^\W_]+)[\^\_\{]+(\d)\}\)")
    subs.append("DX(u,\g<1>+\g<2>-1+I,dx)")
    
    regexs.append(r"\\partial_\{2\}\(([^\W_]+)[\^\_\{]+(\d)\}\)")
    subs.append("DY(u,\g<1>+\g<2>-1+I,dy)")
    
    regexs.append(r"\\partial_\{3\}\(([^\W_]+)[\^\_\{]+(\d)\}\)")
    subs.append("DZ(u,\g<1>+\g<2>-1+I,dz)")
    
    # Scalars
    
    regexs.append(r"\\partial_\{1\}\(([^\W_]+)\)")
    subs.append("DX(u,\g<1>+I,dx)")
    
    regexs.append(r"\\partial_\{2\}\(([^\W_]+)\)")
    subs.append("DY(u,\g<1>+I,dy)")
    
    regexs.append(r"\\partial_\{3\}\(([^\W_]+)\)")
    subs.append("DZ(u,\g<1>+I,dz)")

    # Non-derivative Terms
    
    regexs.append(r"([^\W_]+)[\^\_\{]+(\d)[\s\}\{\^\_]+(\d)\}")
    subs.append("u[\g<1>+3*(\g<2>-1)+\g<3>-1+I]")
    
    regexs.append(r"([^\W_]+)[\^\_\{]+(\d)\}")
    subs.append("u[\g<1>+\g<2>-1+I]")
    
    regexs.append(r" ([^\W_^I]+) ")
    subs.append("u[\g<1>+I]")
    
    regexs.append(r" I ")
    subs.append(" sys->im ")
    
    
    for i in range(len(regexs)):
        line = re.sub(regexs[i],subs[i],line)
    
    for key in sols_dict.keys():
        line = re.sub(key,str(sols_dict[key]), line)
    
    return line
    
def Evaluate_Indices(line,i_min=0):
    line = ''.join([c for c in line if c != " "])
    number_chars = "0123456789+-/*()"
    I_ind = line.find("I",i_min)
    if I_ind == -1: return line
    i = 1
    while line[I_ind-i] in number_chars:
        i += 1

    num_str = str(eval(line[I_ind - i+1:I_ind-1]))
    line = line[:I_ind-i+1] + num_str + line[I_ind-1:]
    return Evaluate_Indices(line,I_ind-i+1+len(num_str)+2)
    
def Evaluate_Temp_Indices(line,i_min=0):
    Bra_Ind = line.find("[",i_min)
    if Bra_Ind == -1: return line
    i = 1
    while line[Bra_Ind+i] != "]":
        i+=1
    num_str = line[Bra_Ind+1:Bra_Ind+i]
    if "I" in num_str:
        return Evaluate_Temp_Indices(line,Bra_Ind+i+1)
    line = line[:Bra_Ind+1] + str(eval(num_str)) + line[Bra_Ind+i:]
    return Evaluate_Temp_Indices(line,Bra_Ind + i - len(num_str))
    

In [19]:
def Step_Equations(line):
    equal_pos = line.find("=")
    rhs_string = line[equal_pos+1:len(line)]
    lhs_string = line[0:equal_pos]

    lhs_string = re.sub(r"\{(\d), (\d)\}","u[3*(\g<1>-1)+(\g<2>-1) + I]",lhs_string)
    
    return f"d{lhs_string} = {rhs_string};"
    
    
def Constraint_Equation(line):
    equal_pos = line.find("=")
    if equal_pos == -1:
        return f"G += {line};"
    rhs_string = line[equal_pos+1:len(line)]
    lhs_string = line[0:equal_pos]
    
    Indices = re.findall(r"\d",lhs_string)
    index_str = ','.join(Indices)
    return f"G[{index_str}-1] = {rhs_string};"

def Write_Evolution_Equations(lines :[str],sols_dict: dict) -> None:
    f = open(f"{list(sols_dict.keys())[0]}_evolutions.txt","w+")
    
    
    others = []
    for i in range(len(lines)):
        lines[i] = add_mult_symbols(lines[i])
        lines[i] = apply_regexs_2(lines[i],sols_dict)
        lines[i] = Step_Equations(lines[i])
        lines[i] = Evaluate_Indices(lines[i])
        lines[i] = Evaluate_Temp_Indices(lines[i])
        lines[i] = re.sub(r"(\d)/(\d)","(\g<1>.0f/\g<2>.0f)",lines[i])
        lines[i] = lines[i].replace("sys->im","IM")
        lines[i] = lines[i].replace("du[","u_rhs[")
        lines[i] = lines[i].replace("S[","E[")
        others.append(lines[i])
        
    for other in others:
        f.write(other)
        f.write("\n")
    f.close()
    
def Write_Constraint_Equations(lines,sols_dict):
    f = open(f"{list(sols_dict.keys())[0]}_constraints.txt","w+")
    
    writes = []
    for i in range(len(lines)):
        lines[i] = lines[i]
        lines[i] = add_mult_symbols(lines[i])
        lines[i] = apply_regexs_2(lines[i],sols_dict)
        lines[i] = Constraint_Equation(lines[i])
        lines[i] = re.sub(r"(D[XYZ])","sys->\g<1>",lines[i])
        lines[i] = Evaluate_Indices(lines[i])
        lines[i] = Evaluate_Temp_Indices(lines[i])
        lines[i] = re.sub(r"(\d)/(\d)","(\g<1>.0f/\g<2>.0f)",lines[i])
        writes.append(lines[i])
        
    for line in writes:
        f.write(line)
        f.write("\n")
    f.close()


### Ashterkar Evolution System

In [5]:
E_dt:= -\epsilon^{i j k} E^{k b} D_{b}{L E^{j a}} - \epsilon^{i j k} \theta^{j} E^{k a} - E^{i b} \partial_{b}{\beta^{a}} + E^{i a} \partial_{b}{\beta^{b}} + \beta^{b} D_{b}{E^{i a}};
product_rule(_)
distribute(_)
substitute(_,$D_{a?}{E^{i? b?}} -> \partial_{a?}{E^{i? b?}} + \epsilon^{i? j k} A^{j}_{a?} E^{k b?}$)
substitute(_,$D_{a}{L} -> \partial_{a}{L}$)
distribute(_)
epsilon_to_delta(_)
expand_delta(_)
distribute(_)
eliminate_kronecker(_)
rename_dummies(_)
evaluate(_)
canonicalise(_)
meld(_)

lines = str(_).split("\n")[1:-1]
for i in range(len(lines)):
    lines[i] = re.sub(r"    \{(\d), (\d)\}","E^{\g<1> \g<2>}",lines[i])
sols_dict = {"E":0, "L":9, "β":9+1, "A":9+1+3, "θ":9+1+3+9}
Write_Evolution_Equations(lines,sols_dict)

${}-\epsilon^{i j k} E^{k b} D_{b}\left(L E^{j a}\right)-\epsilon^{i j k} \theta^{j} E^{k a}-E^{i b} \partial_{b}{\beta^{a}}+E^{i a} \partial_{b}{\beta^{b}}+\beta^{b} D_{b}{E^{i a}}$

Traceback (most recent call last):
  File "/usr/local/lib64/python3.11/site-packages/cadabra2_jupyter/kernel.py", line 56, in do_execute
    self._execute_python(pycode)
  File "/usr/local/lib64/python3.11/site-packages/cadabra2_jupyter/kernel.py", line 110, in _execute_python
    self._sandbox_context(pycode)
  File "/usr/local/lib64/python3.11/site-packages/cadabra2_jupyter/context.py", line 51, in __call__
    exec(code, self._sandbox)
  File "<string>", line 20, in <module>
NameError: name 'Write_Evolution_Equations' is not defined


In [5]:
A_dt:= L \epsilon^{i j k} \epsilon_{a b c} E^{j b} F^{k c} - \epsilon_{a b c} \beta^{b} F^{i c} + \partial_{a}{\theta^{i}} + \epsilon^{i j k} A^{j}_{a} \theta^{k};
evaluate(_)
canonicalise(_)
meld(_)

lines = str(_).split("\n")[1:-1]
for i in range(len(lines)):
    lines[i] = re.sub(r"    \{(\d), (\d)\}","A^{\g<1> \g<2>}",lines[i])

sols_dict = {"E":0, "L":9, "β":9+1, "A":9+1+3, "θ":9+1+3+9}
Write_Evolution_Equations(lines,sols_dict)

${}L \epsilon^{i j k} \epsilon_{a b c} E^{j b} F^{k c}-\epsilon_{a b c} \beta^{b} F^{i c}+\partial_{a}{\theta^{i}}+\epsilon^{i j k} A^{j}\,_{a} \theta^{k}$

In [14]:
F_def:= 1/2 \epsilon^{a b c} (\partial_{b}{A^{i}_{c}} - \partial_{c}{A^{i}_{b}} + \epsilon^{i j k} A^{j}_{b} A^{k}_{c});
evaluate(_)

lines = str(_).split("\n")[1:-1]
for i in range(len(lines)):
    lines[i] = re.sub(r"    \{(\d), (\d)\}","F^{\g<1> \g<2>}",lines[i])
sols_dict = {"E":0, "L":9, "β":9+1, "A":9+1+3, "θ":9+1+3+9}
Write_Evolution_Equations(lines,sols_dict)

${}\frac{1}{2}\epsilon^{a b c} \left(\partial_{b}{A^{i}\,_{c}}-\partial_{c}{A^{i}\,_{b}}+\epsilon^{i j k} A^{j}\,_{b} A^{k}\,_{c}\right)$

### Ashtekar Constraint

In [9]:
G:= \partial_{a}{E^{i a}} + \epsilon^{i j k} A^{j}_{a} E^{k a};
evaluate(_)
lines = str(_).split("\n")[1:-1]
sols_dict = {"E":0, "L":9, "β":9+1, "A":9+1+3, "θ":9+1+3+9}
Write_Constraint_Equations(lines,sols_dict)

${}\partial_{a}{E^{i a}}+\epsilon^{i j k} A^{j}\,_{a} E^{k a}$

### Lapse Calculation

In [23]:
N:= 1/6 \epsilon^{i j k} \epsilon_{a b c} E^{i a} E^{j b} E^{k c};
evaluate(_)
sort_product(_)
lines = [str(_)]

sols_dict = {"E":0, "L":9, "β":9+1, "A":9+1+3, "θ":9+1+3+9}
Write_Constraint_Equations(lines,sols_dict)

${}\frac{1}{6}\epsilon^{i j k} \epsilon_{a b c} E^{i a} E^{j b} E^{k c}$

### Pure Connection Evolution

#### $\psi^{i j}$ Evolution

In [19]:
Psi_dt:= \beta^{a} \partial_{a}{\psi^{i j}} + I L \epsilon^{j k l} S^{k a} D_{a}{\psi^{i l}} - \epsilon^{i k l} \theta^{k} \psi^{l j} - \epsilon^{j k l} \theta^{k} \psi^{l i};
#Psi_dt:= 1/2 I L \epsilon^{j k l} E^{k a} D_{a}{\psi^{i l}} + 1/2 I L \epsilon^{i k l} S^{k a} D_{a}{\psi^{j l}} - \epsilon^{i k l} \theta^{k} \psi^{l j} - \epsilon^{j k l} \theta^{k} \psi^{l i};
substitute(_,$D_{a}{\psi^{i j}} -> \partial_{a}{\psi^{i j}} + \epsilon^{i k l} A^{k}_{a} \psi^{l j} + \epsilon^{j k l} A^{k}_{a} \psi^{i l}$)
distribute(_)
rename_dummies(_)
evaluate(_)
sort_product(_)
canonicalise(_)
meld(_)

lines = str(_).split("\n")[1:-1]
sols_dict = {"ψ":0, "L":9, "β":9+1, "A":9+1+3, "θ":9+1+3+9}
Write_Evolution_Equations(lines,sols_dict)

${}\beta^{a} \partial_{a}{\psi^{i j}}+I L \epsilon^{j k l} S^{k a} D_{a}{\psi^{i l}}-\epsilon^{i k l} \theta^{k} \psi^{l j}-\epsilon^{j k l} \theta^{k} \psi^{l i}$

#### $A^i_a$ Evolution

In [20]:
#A_dt:= D_{a}{\theta^{i}} - I L \epsilon^{i j k} \epsilon_{a b c} S^{j b} F^{k c}- \epsilon_{a b c} \beta^{b} F^{i c};
A_dt:= D_{a}{\theta^{i}} + I L/2 \psi^{i j} \epsilon_{a b c} \epsilon^{j k l} S^{k b} S^{l c} - \epsilon_{a b c} \beta^{b} F^{i c};
substitute(_,$D_{a}{\theta^{i}} -> \partial_{a}{\theta^{i}} + \epsilon^{i j k} A^{j}_{a} \theta^{k}$)
evaluate(_)
canonicalise(_)
meld(_)

lines = str(_).split("\n")[1:-1]
for i in range(len(lines)):
    lines[i] = re.sub(r"    \{(\d), (\d)\}","A^{\g<1> \g<2>}",lines[i])
sols_dict = {"ψ":0, "L":9, "β":9+1, "A":9+1+3, "θ":9+1+3+9}
Write_Evolution_Equations(lines,sols_dict)

${}D_{a}{\theta^{i}}+\frac{1}{2}I L \psi^{i j} \epsilon_{a b c} \epsilon^{j k l} S^{k b} S^{l c}-\epsilon_{a b c} \beta^{b} F^{i c}$

### $S^{i a}$ Definition

In [15]:
S_def:= 1/2 \epsilon^{i k l} \epsilon^{j m n} \psi^{k m} \psi^{l n} F^{j a};
evaluate(_)
sort_product(_)
canonicalise(_)
meld(_)

lines = str(_).split("\n")[1:-1]

sols_dict = {"ψ":0, "L":9, "β":9+1, "A":9+1+3, "θ":9+1+3+9}
Write_Evolution_Equations(lines,sols_dict)

${}\frac{1}{2}\epsilon^{i k l} \epsilon^{j m n} \psi^{k m} \psi^{l n} F^{j a}$

### Constraint Equation

#### $\psi^{i j}$ Gauss Consrtaint

In [103]:
C_phi:= D_{a}{\psi^{i j}} S^{j a};
substitute(_,$D_{a}{\psi^{i j}} -> \partial_{a}{\psi^{i j}} + \epsilon^{i k l} A^{k}_{a} \psi^{l j} + \epsilon^{j k l} A^{k}_{a} \psi^{i l}$)
distribute(_)
canonicalise(_)
meld(_)
evaluate(_)
sort_product(_)
canonicalise(_)
meld(_)

lines = str(_).split("\n")[1:-1]
sols_dict = {"ψ":0, "L":9, "β":9+1, "A":9+1+3, "θ":9+1+3+9}
Write_Constraint_Equations(lines,sols_dict)

${}D_{a}{\psi^{i j}} S^{j a}$

### Determiant Calcuation

In [104]:
Det_phi:= 1/6 \epsilon^{i j k} \epsilon^{l m n} \psi^{i l} \psi^{j m} \psi^{k n};
evaluate(_)
sort_product(_)
_;
canonicalise(_)
meld(_)

lines = [str(_)]
sols_dict = {"ψ":0, "L":9, "β":9+1, "A":9+1+3, "θ":9+1+3+9}
Write_Constraint_Equations(lines,sols_dict)

${}\frac{1}{6}\epsilon^{i j k} \epsilon^{l m n} \psi^{i l} \psi^{j m} \psi^{k n}$

${}\psi^{1 1} \psi^{2 2} \psi^{3 3}-\psi^{1 1} \psi^{2 3} \psi^{3 2}-\psi^{1 2} \psi^{2 1} \psi^{3 3}+\psi^{1 3} \psi^{2 1} \psi^{3 2}+\psi^{1 2} \psi^{2 3} \psi^{3 1}-\psi^{1 3} \psi^{2 2} \psi^{3 1}$

### $det(F)$ Defition

In [105]:
Det_F:= 1/6 \epsilon^{i j k} \epsilon_{a b c} F^{i a} F^{j b} F^{k c};
evaluate(_)
sort_product(_)
_;
lines = [str(_)]
sols_dict = {"ψ":0, "L":9, "β":9+1, "A":9+1+3, "θ":9+1+3+9}
Write_Constraint_Equations(lines,sols_dict)

${}\frac{1}{6}\epsilon^{i j k} \epsilon_{a b c} F^{i a} F^{j b} F^{k c}$

${}F^{1 1} F^{2 2} F^{3 3}-F^{1 1} F^{2 3} F^{3 2}-F^{1 2} F^{2 1} F^{3 3}+F^{1 2} F^{2 3} F^{3 1}+F^{1 3} F^{2 1} F^{3 2}-F^{1 3} F^{2 2} F^{3 1}$

### $\psi^{i j}$ Modified Evolution

In [100]:
Psi_dt:= 1/2 I L \epsilon^{j k l} S^{k a} D_{a}{\psi^{i l}} + 1/2 I L \epsilon^{i k l} S^{k a} D_{a}{\psi^{j l}} + 3/2 I L S^{i a} D_{a}{\phi^{j}} + 3/2 I L S^{j a} D_{a}{\phi^{i}} - I L \delta^{i j} S^{k a} D_{a}{\phi^{k}} - \epsilon^{i k l} \theta^{k} \psi^{l j} - \epsilon^{j k l} \theta^{k} \psi^{l i};
substitute(_,$D_{a}{\psi^{i j}} -> \partial_{a}{\psi^{i j}} + \epsilon^{i k l} A^{k}_{a} \psi^{l j} + \epsilon^{j k l} A^{k}_{a} \psi^{i l}$)
substitute(_,$D_{a}{\phi^{i}} -> \partial_{a}{\phi^{i}} + \epsilon^{i j k} A^{j}_{a} \phi^{k}$)
distribute(_)
evaluate(_)
sort_product(_)
canonicalise(_)
meld(_)

lines = str(_).split("\n")[1:-1]
sols_dict = {"ψ":0, "L":9, "β":9+1, "A":9+1+3, "θ":9+1+3+9, "φ":9+1+3+9+3}
Write_Evolution_Equations(lines,sols_dict)

${}\frac{1}{2}I L \epsilon^{j k l} S^{k a} D_{a}{\psi^{i l}}+\frac{1}{2}I L \epsilon^{i k l} S^{k a} D_{a}{\psi^{j l}}+\frac{3}{2}I L S^{i a} D_{a}{\phi^{j}}+\frac{3}{2}I L S^{j a} D_{a}{\phi^{i}}-I L \delta^{i j} S^{k a} D_{a}{\phi^{k}}-\epsilon^{i k l} \theta^{k} \psi^{l j}-\epsilon^{j k l} \theta^{k} \psi^{l i}$

In [102]:
Phi_dt:= -I L 1/2 \epsilon^{i j k} S^{j a} D_{a}{\phi^{k}} -I L 1/2 S^{j a} D_{a}{\psi^{i j}} + \beta^{a} D_{a}{\phi^{i}} - \epsilon^{i j k} \theta^{j} \phi^{k};
substitute(_,$D_{a}{\psi^{i j}} -> \partial_{a}{\psi^{i j}} + \epsilon^{i k l} A^{k}_{a} \psi^{l j} + \epsilon^{j k l} A^{k}_{a} \psi^{i l}$)
substitute(_,$D_{a}{\phi^{i}} -> \partial_{a}{\phi^{i}} + \epsilon^{i j k} A^{j}_{a} \phi^{k}$)
distribute(_)
evaluate(_)
sort_product(_)

lines = str(_).split("\n")[1:-1]
for i in range(len(lines)):
    lines[i] = re.sub(r"    \{(\d)\}","φ^{\g<1>}",lines[i])
    
sols_dict = {"ψ":0, "L":9, "β":9+1, "A":9+1+3, "θ":9+1+3+9, "φ":9+1+3+9+3}
Write_Evolution_Equations(lines,sols_dict)

${} - \frac{1}{2}I L \epsilon^{i j k} S^{j a} D_{a}{\phi^{k}} - \frac{1}{2}I L S^{j a} D_{a}{\psi^{i j}}+\beta^{a} D_{a}{\phi^{i}}-\epsilon^{i j k} \theta^{j} \phi^{k}$

### Modified Equations of Motion

This section contains code to generate the equations of motion for the modified Weyl Evolution system
where the extra 3 parameters have be absorbed into the self-dual Weyl curvature.

$\Psi^{ij} = \psi^{(ij)} - \epsilon^{ijk} \phi^k$

where $\Psi^{ij}$ is now our fundamental variable. 

Firstly one needs to calculate the metric triad via

$\tilde{E}^{ia} = \frac{1}{\psi^{ij}} \tilde{F}^{ja}$

where $\psi^{ij} = \frac{\Psi^{ij} + \Psi^{ji}}{2}$

In [11]:
E_part:= 1/8 \epsilon^{i k l} \epsilon^{j m n} (\Psi^{k m} +\Psi^{m k}) (\Psi^{l n} +\Psi^{n l}) F^{j a};
epsilon_to_delta(_)
expand_delta(_)
distribute(_)
eliminate_kronecker(_)
meld(_)
rename_dummies(_)
substitute(_,$\Psi^{k k} = 0 $)
canonicalise(_)
evaluate(_)

lines = str(_).split("\n")[1:-1]

sols_dict = {"ψ":0, "L":9, "β":9+1, "A":9+1+3, "θ":9+1+3+9}
Write_Constraint_Equations(lines,sols_dict)

${}\frac{1}{8}\epsilon^{i k l} \epsilon^{j m n} \left(\Psi^{k m}+\Psi^{m k}\right) \left(\Psi^{l n}+\Psi^{n l}\right) F^{j a}$

In [22]:
det_Psi:= 1/6 \epsilon^{i j k} \epsilon^{l m n} \Psi^{i l} \Psi^{j m} \Psi^{k n};
epsilon_to_delta(_)
expand_delta(_)
distribute(_)
eliminate_kronecker(_)
substitute(_,$\Psi^{k k} = 0$)
rename_dummies(_)
meld(_)
evaluate(_)

lines = [str(_)]
sols_dict = {"ψ":0, "L":9, "β":9+1, "A":9+1+3, "θ":9+1+3+9}
Write_Constraint_Equations(lines,sols_dict)

${}\frac{1}{6}\epsilon^{i j k} \epsilon^{l m n} \Psi^{i l} \Psi^{j m} \Psi^{k n}$

### $\psi^{i j} = \psi^{(i j)} - \epsilon^{i j k} \phi^{k}$

In [20]:
Phi_dt:= I L/2 ( \epsilon^{j l k} \delta^{i m} + \epsilon^{j m k} \delta^{i l} + \epsilon^{i l k} \delta^{j m} + \epsilon^{i m k} \delta^{j l} - \epsilon^{i j m} \delta^{l k} )S^{l a} D_{a}{\psi^{m k}}) - \epsilon^{i k l} \theta^{k} \psi^{l j} - \epsilon^{j k l} \theta^{k} \psi^{l i} + \alpha (\psi^{i j} - \psi^{j i});
distribute(_)
substitute(_,$D_{a}{\psi^{i j}} -> \partial_{a}{\psi^{i j}} + \epsilon^{i k l} A^{k}_{a} \psi^{l j} + \epsilon^{j k l} A^{k}_{a} \psi^{i l}$)
distribute(_)
evaluate(_)
sort_product(_)

lines = str(_).split("\n")[1:-1]
for i in range(len(lines)):
    lines[i] = re.sub(r"    \{(\d), (\d)\}", "ψ^{\g<1> \g<2>}",lines[i])
    lines[i] = str(lines[i])

sols_dict = {"ψ":0, "L":9, "β":9+1, "A":9+1+3, "θ":9+1+3+9, "α": -1}
Write_Evolution_Equations(lines,sols_dict)

${}\frac{1}{2}I L \left(\epsilon^{j l k} \delta^{i m}+\epsilon^{j m k} \delta^{i l}+\epsilon^{i l k} \delta^{j m}+\epsilon^{i m k} \delta^{j l}-\epsilon^{i j m} \delta^{l k}\right) S^{l a} D_{a}{\psi^{m k}}-\epsilon^{i k l} \theta^{k} \psi^{l j}-\epsilon^{j k l} \theta^{k} \psi^{l i}+\alpha \left(\psi^{i j}-\psi^{j i}\right)$